In [10]:
from prediction import *
import datetime

In [11]:
forecast_weather = pd.read_csv('../../Data\\capstone\\weather/solar_forecast_weather.csv')
solar = pd.read_csv('../../Data\\capstone\\weather/solar_power_2204.csv')
real_weather = pd.read_csv('../../Data/capstone/total_eda_real.csv')
solar_2 = pd.read_csv('../../Data/capstone/발전소/하동_eda.csv')

def time_change(data, Forecast_time):
    """
    data = forecast가 있는 데이터
    Forecast_time = 예보시간

    return = 예보시간을 기준으로 데이터를 정렬한 데이터
    """
    data[Forecast_time] = pd.to_datetime(data[Forecast_time])
    data['real_time'] = data.apply(lambda row: row['Forecast time'] + datetime.timedelta(hours=row['forecast']), axis=1)
    forecast_weather_eda = data.groupby(['real_time']).mean().drop('forecast', axis=1)
    forecast_weather_eda.reset_index(inplace=True)
    return forecast_weather_eda



In [12]:
solar['datetime'] = pd.to_datetime(solar['datetime'])
solar.target.replace(65535.0, solar.target.mean(), inplace=True)
solar['month'] = solar.datetime.dt.month
solar['hour'] = solar.datetime.dt.hour
solar['year'] = solar.datetime.dt.year
solar['day'] = solar.datetime.dt.day
solar_eda = solar.groupby(['year','month','day','hour']).sum().reset_index()
solar_eda['date'] = pd.to_datetime(solar_eda[['year', 'month', 'day', 'hour']])

C:\Users\user\AppData\Local\Temp\ipykernel_6824\3394655686.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  solar_eda = solar.groupby(['year','month','day','hour']).sum().reset_index()


In [13]:
eda_forecast = time_change(forecast_weather, 'Forecast time')

reg = real_weather[real_weather['지점명'] =='순천']

eda_weather = reg.drop(['지점', '지점명'], axis=1)
eda_weather['일시'] = pd.to_datetime(eda_weather['일시'])
eda_weather['hour'] = eda_weather['일시'].dt.hour
eda_weather['month'] = eda_weather['일시'].dt.month
eda_weather['year'] = eda_weather['일시'].dt.year

eda_weather.set_index('일시', inplace=True)
total_time = pd.date_range(start='2021-01-01 00:00:00', end='2022-04-30 23:00:00', freq='H')
solar_2.date = pd.to_datetime(solar_2.date)
solar_2.set_index('date', inplace=True)


C:\Users\user\AppData\Local\Temp\ipykernel_6824\412407747.py:15: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  forecast_weather_eda = data.groupby(['real_time']).mean().drop('forecast', axis=1)


In [14]:
x = eda_weather.loc['2021-01-01 00:00:00' : '2022-04-30 23:00:00']
y = solar_2.loc['2021-01-01 00:00:00' : '2022-04-30 23:00:00']
y.drop('hour', axis=1 , inplace=True)
x.drop(['위도','경도'], axis=1, inplace=True)

total = pd.concat([x, y], axis = 1)

complete_date_rng = pd.date_range(start='2021-01-01', end='2022-04-30', freq='H')

# 누락된 시간 찾기
missing_dates = complete_date_rng.difference(total.index)

total_eda = total[(total.index>='2021-09-28') | (total.index <= '2021-09-22')]
total_eda_2 = total_eda[(total_eda.index <= '2022-01-14') | (total_eda.index >= '2022-01-17')]
date_counts = total_eda_2.resample('D').count()
extra_hourly_data = date_counts[date_counts['month'] < 24]


total_eda_2 = total_eda_2[total_eda_2.index != '2022-01-14']
total_eda_2 = total_eda_2[total_eda_2.index != '2021-09-22']

total_eda_2.interpolate(inplace=True, method='linear')

C:\Users\user\AppData\Local\Temp\ipykernel_6824\2731628856.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.drop('hour', axis=1 , inplace=True)
C:\Users\user\AppData\Local\Temp\ipykernel_6824\2731628856.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.drop(['위도','경도'], axis=1, inplace=True)


In [16]:
x_train = total_eda_2.drop('value', axis=1).iloc[:-72]
y_train = total_eda_2['value'].iloc[:-72]

x_test = total_eda_2.drop('value', axis=1).iloc[-72:]
y_test = total_eda_2['value'].iloc[-72:]

In [18]:
pred_model = model(x_train, y_train, x_test, y_test)

In [21]:
pred_model.rf_fit(Grid=False)

RandomForestRegressor(max_depth=5, min_samples_leaf=8)

In [22]:
pred_model.rf_pred(x_test)

array([1.26403364e-04, 1.26403364e-04, 3.10892651e-03, 3.10892651e-03,
       3.10892651e-03, 3.10892651e-03, 7.56062573e-02, 9.67671192e+01,
       5.15008357e+02, 1.46067566e+03, 2.42729470e+03, 2.53535554e+03,
       2.53535554e+03, 2.76617688e+03, 2.96791630e+03, 2.95406179e+03,
       1.76384489e+03, 1.27110468e+03, 5.27526888e+02, 1.15232537e+02,
       6.95763086e+00, 6.95763086e+00, 6.95763086e+00, 8.04683987e+00,
       1.26403364e-04, 1.26403364e-04, 1.26403364e-04, 1.26403364e-04,
       3.10892651e-03, 3.10892651e-03, 7.56062573e-02, 1.18588815e+02,
       3.93642456e+02, 6.86501847e+02, 8.58867377e+02, 8.86270346e+02,
       1.54374599e+03, 1.35172084e+03, 1.57998080e+03, 1.57998080e+03,
       1.60002442e+03, 1.10499646e+03, 1.07619014e+02, 2.00811490e+01,
       6.95763086e+00, 6.95763086e+00, 6.95763086e+00, 6.95763086e+00,
       1.26403364e-04, 1.26403364e-04, 1.26403364e-04, 1.26403364e-04,
       1.26403364e-04, 1.26403364e-04, 2.54677028e-02, 1.09699856e+02,
      